# Einstein summation convention and np.einsum examples

The Einstein summation convention over repeated indices is that we drop to write summation symbol. The result can be scalars, vectors, matrices or tensors and is determined by the indices that are not summed over.

This is best demonstrated through a couple of examples. 

The trace of a matrix can be written using the Einstein summation as follows
\begin{equation}
tr(A) = \sum_k A_k^k = A_k^k
\end{equation}

The einsum function in numpy can be used to evaluate expressions using the Einstein summation convention. One specifies which indices to sum over and the number of indices in the output.

In [6]:
import numpy as np

np.random.seed(1)

N = 10
A = np.eye(N)
trA = np.einsum("kk->",A)
print("tr(A): %g" % trA)

tr(A): 10


The matrix-vector product
\begin{equation}
\mathbf{b}_i = (A\mathbf{x})_i = \sum_{k} A_{ik}\mathbf{x}_k = A_{ik}\mathbf{x}_k.  
\end{equation}
Note that in this example there is one index that repeats and that do not. This implies that result is a vector.

In [7]:
A = np.random.rand(10,10)
x = np.random.rand(10)

b = np.einsum("ik,k->i",A,x)
print(b)

[1.95050321 2.25619465 3.68157469 3.66583057 2.23653399 2.37684215
 3.12093308 2.82446368 4.19281221 2.8266817 ]


The matrix-matrix product
\begin{equation}
(C)_{ij} = (AB)_{ij} = \sum_k a_{ik}b_{kj} = a_{ik}b_{kj}. 
\end{equation}
Here we have to indices which are not repeated which implies that the result is a matrix.

In [9]:
A = np.random.rand(2,2)
B = np.random.rand(2,2)
C = np.einsum("ik,kj->ij",A,B)
print(C)

[[0.12701727 0.24214633]
 [0.02310492 0.019018  ]]


Next we look at a couple of examples directly relevant for our purposes. It should be noted that einsum does not 
use the convention that $i,j,k,\dots$ index hole states and $a,b,c,\dots$ index particle states. However this can be solved by using slices. 

In the following examples $N$ denote the number of holes and $L$ is the total number of single particle states and $M=L-N$ is the number of particle states. We construct random matrices and tensors do demonstrate computation of some central quantities.

In [10]:
N = 4
L = 40
M = L-N 

h = np.random.rand(L,L)
v = np.random.rand(L,L,L,L)

hole = slice(0,N) #hole indices
part = slice(N,L) #particle indices

The reference energy
\begin{equation}
E_\text{ref} = \sum_{i=1}^N h_i^i + \frac{1}{2} \sum_{ij=1}^N u^{ij}_{ij} = h_i^i + \frac{1}{2}u^{ij}_{ij}
\end{equation}

In [11]:
#Note the use of the slices to only sum over particle indices
Eref = np.einsum("ii->",h[hole,hole]) + 0.5*np.einsum("ijij->",v[hole,hole,hole,hole])
print(Eref)

6.893387986786276


The fock matrix
\begin{align}
h_{\alpha \beta}^{\text{HF}} &= \langle \alpha|\hat{h}|\beta \rangle + \sum_j \sum_{\gamma \delta = 1}^L C^*_{j \gamma} C_{j \delta} \langle \alpha \gamma | \hat{v} | \beta \delta \rangle_\text{AS} \\
&= h_{\alpha \beta}^{\text{HF}} \equiv \langle \alpha|\hat{h}|\beta \rangle + \sum_{\gamma \delta = 1}^N D_{\gamma \delta} \langle \alpha \gamma | \hat{v} | \beta \delta \rangle_\text{AS} 
\end{align}
where 
\begin{equation}
D_{\gamma \delta} = \sum_{j=1}^N C^*_{j \gamma} C_{j \delta}
\end{equation}
Note that $\gamma,\delta$ sum over all indices while $j$ only sums over particle indices.

In [12]:
C   = np.random.rand(L,L)
D   = np.einsum("jg,jd->gd",np.conj(C[hole,:]),C[hole,:])
hHF = h + np.einsum("gd,agbd->ab",D,v)
print(hHF.shape)

(40, 40)


Finally, lets take an example from the CCD amplitude equation 
\begin{equation}
\sum_{klcd} \langle kl|\hat{v}|cd \rangle t^{ac}_{ik} t^{bd}_{lj}.
\end{equation}
The amplitudes have dimension $M\times M \times N \times N$. Note also that in contrast to the other examples we now have 
three quantities in the sum. Here the result is a new tensor of dimension $M\times M \times N \times N$.

In [13]:
import time
t = np.random.rand(M,M,N,N)

t0 = time.time()
t_new = np.einsum("klcd,acik,bdlj->abij",v[hole,hole,part,part],t,t)
t1 = time.time()
print(t_new.shape)
print("Time: %g" % (t1-t0))

(36, 36, 4, 4)
Time: 10.1975


The above expression scales more poorly than necessary if computed naively. Note that for every $a,b,i,j$ we sum over $k,l,c,d$ resulting in an $O(M^4 N^4$ scaling. However we can speed up the computation by reordering the expression as follows 
\begin{equation}
\sum_{klcd} \langle kl|\hat{v}|cd \rangle t^{ac}_{ik} t^{bd}_{lj} = \sum_{ld} \chi^{ad}_{il} t^{bd}_{jl}
\end{equation}
where 
\begin{equation}
  \chi^{ad}_{il} = \langle kl|\hat{v}|cd \rangle t^{ac}_{ik}
\end{equation}

Alternatively we can use the optimize argument in einsum which tries to optimize the sum for us (and very likely does a good job at that).

In [14]:
chi_adil = np.einsum("klcd,acik->adil",v[hole,hole,part,part],t)
t0 = time.time()
t_new = np.einsum("adil,bdjl->abij",chi_adil,t)
t1 = time.time()
print("Time: %g" % (t1-t0))

t0 = time.time()
t_new = np.einsum("klcd,acik,bdlj->abij",v[hole,hole,part,part],t,t,optimize=True)
t1 = time.time()
print(t_new.shape)
print("Time: %g" % (t1-t0))

Time: 0.0614765
(36, 36, 4, 4)
Time: 0.0503128
